In [1]:
import os 
import cv2
import numpy as np
import neuroglancer
from skimage import io, measure
import matplotlib.pyplot as plt

viewer = neuroglancer.Viewer()
print(viewer)

http://127.0.0.1:34105/v/a609b026c6c4a1bde34373996fc4e4c0945de020/


In [2]:
def mask_to_shell(mask):
    sub_contours = measure.find_contours(mask, 1)

    sub_shells = []
    for sub_contour in sub_contours:
        sub_contour.T[[0, 1]] = sub_contour.T[[1, 0]]
        pts = sub_contour.astype(np.int32).reshape((-1, 1, 2))

        sub_shell = np.zeros(mask.shape, dtype='uint8')
        sub_shell = cv2.polylines(sub_shell, [pts], True, 1, 5, lineType=cv2.LINE_AA)
        sub_shells.append(sub_shell)
    shell = np.array(sub_shells).sum(axis=0)
    
    return shell

In [3]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/masks/MD594/aligned'
files = sorted(os.listdir(VOL_DIR))

volume = []
for file in files:
    tif = io.imread(os.path.join(VOL_DIR, file))
    tif = mask_to_shell(tif)
    volume.append(tif)
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)

In [4]:
layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[460, 460, 20000]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
#     s.layers.clear()
    s.layers['layer'] = layer